<h2>Import Libraries

In [3]:
import os
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import random

<h2>Load Model

In [4]:
model_path = os.path.join("model", "facetracker.h5")
if os.path.exists(model_path):
    model = load_model(model_path)

<h2>Make Prediction

In [ ]:
def predict(img_path):
    # Preprocessing Image
    byte_img = tf.io.read_file(img_path)
    img = tf.io.decode_jpeg(byte_img)
    img = tf.image.resize(img, (120,120))
    img = img/255

    # Making Prediction
    yhat = model.predict(np.expand_dims(img, 0))
    cls = yhat[0][0][0]
    coords = yhat[1][0]
    print("Class:", cls)
    print("Coordinates:", coords)

    # Plotting Image
    fig, axs = plt.subplots(ncols=2, figsize=(10,10))
    axs[0].imshow(img)
    x1, y1, x2, y2 = coords * 120
    width = x2 - x1
    height = y2 - y1
    rectangle = patches.Rectangle((x1, y1), width, height, linewidth=2, edgecolor="r", facecolor="None")
    axs[1].imshow(img)
    axs[1].add_patch(rectangle)

In [ ]:
test_images_path = os.path.join("data", "test", "images")
test_images_list = os.listdir(test_images_path)

In [ ]:
image_name = random.choice(test_images_list)
image_path = os.path.join(test_images_path, image_name)
predict(image_path)

<h2>Live Demo

In [1]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _ , frame = cap.read()
    frame = frame[50:500, 50:500,:]

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120,120))

    yhat = model.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]

    if yhat[0] > 0.5:
        # Controls the main rectangle
        cv2.rectangle(frame,
                      tuple(np.multiply(sample_coords[:2], [450,450]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [450,450]).astype(int)),
                            (255,0,0), 2)
        # Controls the label rectangle
        cv2.rectangle(frame,
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                    [0,-30])),
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                    [80,0])),
                            (255,0,0), -1)

        # Controls the text rendered
        cv2.putText(frame, 'face', tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                               [0,-5])),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

    cv2.imshow('FaceTrack', frame)

    key = cv2.waitKey(1)

    if key & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined